通过mmseg的segformer中的model.encode_decode()获取了800张cityscapes的特征图，每张特征图的形状为（19，1024，1024），请问，如何用tsne可视化这800张特征图的分布，其中不同的特征用不同的颜色表示，cityscapes共有19个类。

In [ ]:
import mmcv
from mmcv import Config
from mmseg.apis import inference_segmentor, init_segmentor
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from mmseg.datasets import build_dataloader, build_dataset

In [ ]:
# initialize the segmentor
cfg = Config.fromfile('configs/segformer/segformer_mit-b0_8x1_1024x1024_160k_cityscapes.py')
checkpoint_file = 'checkpoint/segformer_mit-b0_8x1_1024x1024_160k_cityscapes_20211208_101857-e7f88502.pth'
model = init_segmentor(cfg, checkpoint_file, device='cuda:0')

# load the dataset
data_dir = 'data/cityscapes/leftImg8bit/train'
#img_infos = mmcv.scan_images(data_dir, recursive=True)
dataset = build_dataset(cfg.data.train)
dataloader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=1,
    dist=False,
    shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
model.eval()
model.to(device)
feat_maps = []
for i, data in enumerate(dataloader):
  img_metas = data['img_metas'].data
  images = data['img'].data[0].to(device)
  img_path = data['img_metas'].data[0][0].get('filename')
  out = model.encode_decode(images, img_metas)#out.shape (1,19,1024,1024)
  out = out.squeeze()
  feat_map = out.reshape(19,-1)
  feat_map = feat_map.cpu().detach().numpy()
  feat_maps.append(feat_map)
  if i == 200:
    break

通过mmseg的segformer中的model.encode_decode()获取了800张cityscapes的特征图，每张特征图的形状为（1，19，1024，1024），请问，如何用tsne可视化这800张特征图的中特征分布情况，其中不同的特征用不同的颜色表示，cityscapes共有19个类




针对您的数据，您需要对每个类别分别进行t-SNE降维，然后将每个类别的降维结果合并起来进行可视化。

以下是一个简单的代码示例，假设您的特征图中的19个类别分别为0到18：


我有个问题就是我的特征图里面的值都是浮点数

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 对每个类别进行t-SNE降维处理
tsne = TSNE(n_components=2)

X_all = np.empty((0, 19*1024*1024))
labels_all = np.empty((0,), dtype=int)

for label in range(19):
    # 选取特定类别的特征图
    features = feature_maps[:, label, :, :]
    
    # 将所有特征图展平成一维向量
    flatten_features = []
    for feature_map in features:
        flatten_features.append(feature_map.reshape(-1))

    X = np.array(flatten_features)

    # 对当前类别进行t-SNE降维
    X_tsne = tsne.fit_transform(X)
    
    # 将降维结果和对应的标签合并到总数组中
    X_all = np.concatenate((X_all, X_tsne))
    labels_all = np.concatenate((labels_all, np.repeat(label, len(feature_maps))))

# 给每个类别着色
colors = ['r', 'b', 'g', 'c', 'm', 'y', 'k', 'w', 'orange', 'pink', 'purple', 'brown', 'gray', 'olive', 'lime', 'teal', 'navy', 'indigo', 'maroon']
label_colors = [colors[label] for label in labels_all]

# 绘制可视化图像
plt.scatter(X_all[:, 0], X_all[:, 1], c=label_colors)
plt.show()


X_all = np.concatenate((X_all, X_tsne))，ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 19922944 and the array at index 1 has size 2

非常抱歉，我理解有误。您的问题是由于在进行特征图展平后，每张图的特征向量长度为 1910241024 = 19922944，而在对每个类别进行 t-SNE 降维时，t-SNE 方法将特征向量的维度从 19922944 降至 2，导致每个类别的降维结果不再具有相同的维度，无法在 np.concatenate 中进行合并。

为了解决这个问题，您可以在每个类别的 t-SNE 降维之后，将降维后的结果和对应的标签合并为一个元组，将所有元组放入一个列表中，最后使用 plt.scatter 函数将所有点绘制在二维坐标系中。以下是修改后的代码示例：

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 对每个类别进行t-SNE降维处理
tsne = TSNE(n_components=2)

data = []

for label in range(19):
    # 选取特定类别的特征图
    features = feature_maps[:, label, :, :]
    
    # 将所有特征图展平成一维向量
    flatten_features = []
    for feature_map in features:
        flatten_features.append(feature_map.reshape(-1))

    X = np.array(flatten_features)

    # 对当前类别进行t-SNE降维
    X_tsne = tsne.fit_transform(X)
    
    # 将降维结果和对应的标签合并到列表中
    data.append((X_tsne, label))

# 给每个类别着色
colors = ['r', 'b', 'g', 'c', 'm', 'y', 'k', 'w', 'orange', 'pink', 'purple', 'brown', 'gray', 'olive', 'lime', 'teal', 'navy', 'indigo', 'maroon']

# 绘制可视化图像
for X_tsne, label in data:
    label_color = colors[label]
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=label_color)

plt.show()
